In [1]:
import qutip as qt
import numpy as np

# Tracking info of simulations
import tqdm
import time
import warnings

#Saving data
import pandas as pd
import json
import os

In [2]:
from model_building import initialize_1_qubit_system, crate_domain_wall_state, initialize_general_system
from Hamiltonian import Hamiltonian

In [3]:
# Generate standard encoding state
size = 13
alpha = 0.5
beta = 0.5
register = 'Bob'

psi_0 = initialize_1_qubit_system(size, alpha, beta, register)

In [6]:
#generate domain wall state and domain wall chain
dictionary ={"001":0.4,"101":0.2,"111":0.6,"110":0.35}
dictionary= {"1":1}
register = 'Alice'
size = 13
initial_state = crate_domain_wall_state(dictionary, register)
final_state = crate_domain_wall_state(dictionary, register='Bob')

initial_chain = initialize_general_system(size, initial_state, register='Alice')
final_chain = initialize_general_system(size, final_state, register='Bob')

In [ ]:
size = 13
l = 0.02272
J = 0.5
reg_size = len(initial_state.dims[0])

standard_H = Hamiltonian(system_size = size,
                         mode = "standard",
                         lambda_factor = l,
                         register_size = reg_size,
                         global_J = J
                         )